# import

In [ ]:
import os
import re
import pandas as pd
import numpy as np

# Google Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

path = '*******************************************'
os.chdir(path)

In [ ]:
os.getcwd()

In [ ]:
os.listdir()

In [ ]:
path = '*******************************************'
filenames = os.listdir(path)

filenames = list(set(filenames))
print(filenames)

In [ ]:
yogiyo_review = None

for filename in filenames:
    if filename[-3:] == 'csv' :
        temp = pd.read_csv(path + filename, header = None) # sep=','

    elif filename[-3:] == 'txt' :
        temp = pd.read_table(path + filename, header = None) # sep='\t'
    
    if yogiyo_review is None:
        yogiyo_review = temp
        continue

    yogiyo_review = pd.concat([yogiyo_review, temp])

# 인덱스 초기화
yogiyo_review = yogiyo_review.reset_index()

# 자동으로 생성되었던 인덱스 열이 있다면 제거
if 'Unnamed: 0' in yogiyo_review.columns :
    yogiyo_review = yogiyo_review.drop(['Unnamed: 0'], axis=1)
if 'Unnamed: 0.1' in yogiyo_review.columns :
    yogiyo_review = yogiyo_review.drop(['Unnamed: 0.1'], axis=1)
if 'index' in yogiyo_review.columns :
    yogiyo_review = yogiyo_review.drop(['index'], axis=1)


yogiyo_review_df = yogiyo_review.copy()
yogiyo_review_df.columns = ['Total', 'Review']
yogiyo_review_df

In [ ]:
# 불용어 및 특수문자를 없애는 함수
def regex_review(review):
    p = re.compile('[^가-힣 ]')
    result = p.sub('',review)
    result = result.strip()
    result = result.replace('  ', ' ')
    result = result.replace('   ', ' ')
    return result
yogiyo_review_df['regex_review'] = yogiyo_review_df['Review'].apply(regex_review)
yogiyo_review_df

In [ ]:
# 정제한 리뷰의 길이를 센 컬럼 추가
yogiyo_review_df['길이'] = [len(regex_review) for regex_review in yogiyo_review_df.regex_review.to_list()]
yogiyo_review_df

In [ ]:
# 글자 수가 12개 이상, 80개 이하인 것만 남기기
# 너무 길어도 학습이 잘 안 된다.
yogiyo_review_df = yogiyo_review_df[(yogiyo_review_df['길이'] >= 12) & (yogiyo_review_df['길이'] <= 80)]

print(yogiyo_review_df.sort_values(by='길이', axis= 0, ascending=False))

       Total  ...  길이
1329     1.0  ...  80
6961     1.0  ...  80
7073     1.0  ...  80
7629     1.0  ...  80
5349     5.0  ...  80
...      ...  ...  ..
381      5.0  ...  12
1760     5.0  ...  12
12441    3.0  ...  12
4414     5.0  ...  12
1870     1.0  ...  12

[10112 rows x 4 columns]


In [ ]:
yogiyo_review_df.Total.value_counts()

5.0    3571
1.0    3564
3.0    1565
4.0     727
2.0     685
Name: Total, dtype: int64

In [ ]:
yogiyo_review_df[['Total', 'regex_review']].to_csv('/content/drive/MyDrive/bert_classification_kor/preprocessing/data/relabeling_data.txt', sep= '\t', header= None, index = None)

# 평점 병합
1점, 2점, 3점, 4점, 5점 -> 1점, 3점, 5점

In [ ]:
yogiyo_review_df.Total.unique()

array([1., 3., 4., 5., 2.])

In [ ]:
def change(x) :
    if x == 2.0 :
        return 3.0
    elif x == 4.0 :
        return 3.0
    else :
        return x

# yogiyo_review_df['Total'] = yogiyo_review_df['Total'].apply(change)
yogiyo_review_df['Total'] = yogiyo_review_df['Total'].apply(lambda x: 3.0 if x == 2.0 else 3.0 if x == 4.0 else x)
yogiyo_review_df

In [ ]:
yogiyo_review_df.Total.value_counts()

5.0    3571
1.0    3564
3.0    2977
Name: Total, dtype: int64

# 샘플 데이터 생성

In [ ]:
sample_df =pd.DataFrame(columns = ['Total', 'Review', 'regex_review', '길이'])

for rating in yogiyo_review_df['Total'].unique() :
    temporary_df = yogiyo_review_df.query('Total==@rating').sample(n=2977)
    sample_df = pd.concat([sample_df, temporary_df])
    
sample_df = sample_df.reset_index(drop=True)

sample_df = sample_df.sort_values(by='Total', axis= 0)

sample_df = sample_df[['Total', 'regex_review']]
sample_df

In [ ]:
2977*3

8931

In [ ]:
sample_df.to_csv('*******************************************', sep= '\t', header= None, index = None)

# 네이버 맞춤법 검사기로 한번 더 정제

In [ ]:
!pip install git+https://github.com/ssut/py-hanspell.git

  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-05rtua9t
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-05rtua9t
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4868 sha256=72a086b884ad89323af957740ca4fe3547cccbfc4bbba834a570d73373f79826
  Stored in directory: /tmp/pip-ephem-wheel-cache-6wt20xqa/wheels/ab/f5/7b/d4124bb329c905301baed80e2ae45aa14e824f62ebc3ec2cc4
Successfully built py-hanspell


In [ ]:
from hanspell import spell_checker

In [ ]:
review_hanspell = []

for review in sample_df["regex_review"] :
    review_checked = spell_checker.check(review) 
    review_checked = list(review_checked) 
    review_checked = review_checked[2] 
    review_hanspell.append(review_checked)

review_hanspell[:10]

In [ ]:
sample_df['regex_review'] = review_hanspell

In [ ]:
sample_df

In [ ]:
sample_df.to_csv('/content/drive/MyDrive/bert_classification_kor/preprocessing/data/relabeling_data_sampleing_hanspell.txt', sep= '\t', header= None, index = None)